# Introduction

Critical to the prediction of the model, is the medium the cell is being grown in. This notebook will investigate the medium and modify it to a realistic medium. 

In [1]:
import cameo
import pandas as pd
import cobra.io
import escher

In [2]:
model = cobra.io.read_sbml_model('../model/g-thermo.xml')

In [3]:
#for inspiration 
model_e_coli = cameo.load_model('iML1515')

In [4]:
len(model.medium)

151

In [5]:
len(model_e_coli.medium)

24

In [6]:
model.reactions.EX_Biomass_e.bounds = (0,1000)

Here I see that in notebook 17, I added demand reactions instead of exchanges. Therefore these 18 demands are not included in the list of exchanges & medium. So here I first quickly have to modify this before running the script below.
First I will remove all of the demands and then add exchanges for the metabolites without one. 

In [7]:
#make listof of metabolites without exchange, i.e. metabolites in the demand
need_exchanges=[]
for rct in model.demands:
    met =  next(iter(rct.metabolites))
    need_exchanges.append(met)

In [8]:
model.remove_reactions(model.demands)

In [9]:
#add exchange reactions for metabolites
for met in need_exchanges:
    model.add_boundary(met, type = 'exchange', reaction_id = f"EX_{met.id}")

In [10]:
len(model.exchanges)

169

You can see there is a clear difference in the number of components in the medium between the models. Our model has all exchanges as irreversible, and so all components are *in silico* supplied to the model. Therefore this needs to be changed.

I will change this so that the only carbon source is glucose to start with, and we assume a somewhat minimal medium: i.e. no amino acids are supplemented. 

First of all, there are many more exchanges and extracellular metabolites than there are transported into the cell. These will be removed. Then I can re-evaluate how many exchanges too much there are.

To approach the problem, first I will make a list of the exchanges that I think should only be excreted and not taken up. I will run this by Martyn before implementing it.
Then for each reaction one by one i can remove it and check that it doesn't kill biomass accumulation. If so, i will make a note of it and need to look into why it is needed. Then I can remove the medium components that are not required to slim it down. 

In [11]:
extracell_mets = []
for met in model.metabolites:
    met_id = met.id
    if met_id[-1:] in 'e':
        extracell_mets.append(met)
    else:
        continue

In [12]:
len(extracell_mets)

168

In [13]:
#make a list and remove all extracellular metabolites with no transport into the cell
e_met_wo_trans = []
e_met_w_trans =[]
for met in model.metabolites:
    if met in extracell_mets:
        rcts = met.reactions
        if len(met.reactions) == 1:
            rct_id = next(iter(met.reactions)).id
            if rct_id[:2] in 'EX':
                e_met_wo_trans.append(met)
            else:
                print(met)
        else: 
            e_met_w_trans.append(met)
    else:
        continue

pyr_e


Here, pyr_e is printed as it doesn't have a demand yet, so I will add that now. 

In [14]:
model.add_boundary(model.metabolites.pyr_e, type = 'exchange', reaction_id = 'EX_pyr_e')

Reaction identifier,EX_pyr_e
Name,Pyruvate exchange
Memory address,0x02339ddb6608
Stoichiometry,pyr_e <=> Pyruvate <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [15]:
#Pyruvate is not supplied
model.reactions.EX_pyr_e.bounds = (0,1000)

In [16]:
len(e_met_w_trans)

47

In [17]:
len(e_met_wo_trans)

120

In [18]:
#e_met_wo_trans contains all extracellular metabolites without transport. 
#so these metabolites and their exchange reactions can be removed.
for met in e_met_wo_trans:
    rct_id = next(iter(met.reactions)).id
    rct = model.reactions.get_by_id(rct_id)
    model.remove_metabolites(met)
    model.remove_reactions(rct)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\model.py:716: UserWarning:

need to pass in a list



In [19]:
len(model.exchanges)

50

In [22]:
#save&commit cleaning of exchanges
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

In [40]:
#model.medium

From the above code we have made the list of medium to revisit a lot smaller. You can clearly see some things that have to be changed. For example, the strain should (in principle) be able to produce ethanol, acetate, succinate and lactate, but not consume it. Also, we asume a minimal medium with no amino acid supplements, explaining some other changes that should be made (e.g. dlglu_c and asn__L_e).


In [25]:
model.reactions.EX_asn__L_e.bounds = (0,1000)
model.reactions.EX_etoh_e.bounds = (0,1000)
model.reactions.EX_ac_e.bounds = (0,1000)
model.reactions.EX_succ_e.bounds = (0,1000)
model.reactions.EX_lac__D_e.bounds = (0,1000)
model.reactions.EX_lac__L_e.bounds = (0,1000)

In [26]:
#a wrong metabolite, so can be removed
model.remove_metabolites(model.metabolites.succcoa_c)

In [27]:
#this met can be removed as there is already a glutamate intracellularly.
model.remove_metabolites(model.metabolites.dlglu_c)

In [28]:
#oxygen should only be taken up, never producted
model.reactions.EX_o2_e.bounds = (-1000,0)

Thiamin exchange should be investigated further at a later stage. We know from experiments that supplementing minimal medium with Thiamin or TPP significantly increases the growth. And so I will leave this in here for now, and we can take a look at it later.

In [29]:
model.reactions.EX_oh1_e.id = 'EX_h2o_e'

We assume glucose is the only carbon source, as we know on minimal medium, our strain can grow on glucose. So I will remove all other exchanges from the medium by setting the lower bound to 0.

In [30]:
model.reactions.EX_rib__D_e.bounds = (0,1000)
model.reactions.EX_pydx5p_e.bounds = (0,1000)

In [31]:
model.reactions.EX_gal_e.bounds = (0,1000)
model.reactions.EX_man_e.bounds = (0,1000)
model.reactions.EX_arbt_e.bounds = (0,1000)
model.reactions.EX_salcn_e.bounds = (0,1000)

In [32]:
model.reactions.EX_glyc_e.bounds = (0,1000)
model.reactions.EX_tre_e.bounds = (0,1000)
model.reactions.EX_fru_e.bounds = (0,1000)
model.reactions.EX_arab__L_e.bounds = (0,1000)
model.reactions.EX_xyl__D_e.bounds = (0,1000)
model.reactions.EX_mnl_e.bounds = (0,1000)
model.reactions.EX_cellb_e.bounds = (0,1000)
model.reactions.EX_sucr_e.bounds = (0,1000)
model.reactions.EX_sbt__D_e.bounds = (0,1000)

In [33]:
model.reactions.EX_acgam_e.bounds = (0,1000)
model.reactions.EX_malt_e.bounds = (0,1000)
model.reactions.EX_rmn_e.bounds = (0,1000)
model.reactions.EX_melib_e.bounds = (0,1000)
model.reactions.EX_gtbi_e.bounds = (0,1000)
model.reactions.EX_tura_e.bounds = (0,1000)
model.reactions.EX_kdg2_e.bounds = (0,1000)

In [34]:
model.reactions.EX_dglcn5_e.bounds = (0,1000)
model.reactions.EX_mdgp_e.bounds = (0,1000)
model.reactions.EX_tag__D_e.bounds = (0,1000)
model.reactions.EX_amylose_e.bounds = (0,1000)

In [35]:
solution= model.optimize()
solution.objective_value

0.48302692481794285

__Notes__: Reactions that I cannot remove and maintain biomass right now. 
These reactions need further investigation when I've opened the flux constrains as they should and the model is mass balanced.
- EX_gthrd_e
- EX_thm_e
- EX_qh2_e
- EX_nadp_e

In [36]:
#intrestingly TPP is not critical to the model....
model.reactions.EX_thmtp_e.bounds = (0,1000)

In [39]:
model.medium

{'EX_cl_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_glc__D_e': 1000.0,
 'EX_thm_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_o2_e': 1000,
 'EX_co2_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_qh2_e': 1000.0,
 'EX_gthrd_e': 1000.0,
 'EX_nadp_e': 1000.0}

In [38]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

In [41]:
#check on map
solution = model.optimize()
solution

,fluxes,reduced_costs
IDPh,0.000000,-6.440359e-03
PYRACTT,0.000000,2.168404e-19
CAT,0.000000,1.136868e-13
PDHam1hi,0.000000,1.084202e-19
CCP,-0.007222,-0.000000e+00
...,...,...
EX_dglcn5_e,0.000000,0.000000e+00
EX_mdgp_e,0.000000,-1.955109e-03
EX_tag__D_e,0.000000,-6.440359e-03
EX_amylose_e,0.000000,-1.173065e-02
